In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import serial
import wave
from scipy.signal import butter, filtfilt

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*4  # samples per frame
FORMAT = pyaudio.paInt16  # audio format (bytes per sample)
CHANNELS = 1  # single channel for microphone
RATE = 44100  # samples per second

wav_file = wave.open(r"C:\Users\rajka\Downloads\800-1000 alternating.WAV", 'rb')

# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=p.get_format_from_width(wav_file.getsampwidth()),
    channels=wav_file.getnchannels(),
    rate=wav_file.getframerate(),
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
xf = np.linspace(0, RATE, CHUNK)  # frequencies (spectrum)

# create matplotlib figure and axes
fig, ax2 = plt.subplots(figsize=(15, 7))

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')
var = 50
# Define the frequency ranges for the two frequencies
freq_range_1 = (100 - var, 100 + var)  # Frequency range for 100 Hz
freq_range_2 = (5000 - var, 5000 + var)  # Frequency range for 5000 Hz

# Indices corresponding to the chosen frequency ranges
chosen_indices_1 = np.where((xf >= freq_range_1[0]) & (xf <= freq_range_1[1]))[0]
chosen_indices_2 = np.where((xf >= freq_range_2[0]) & (xf <= freq_range_2[1]))[0]

# Chosen frequencies array
chosen_frequencies = [freq_range_1, freq_range_2]

# for measuring frame rate
frame_count = 0
start_time = time.time()

# Read the audio data from the .wav file
data = wav_file.readframes(CHUNK)

latencies = []  # List to store latency values

while len(data) > 0:

    # convert data to integers, make np array, then offset it by 128
    data_int = np.array(struct.unpack(f"{2 * CHUNK}h", data), dtype=np.int16) + 128

    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK]) / np.max(np.abs(yf[0:CHUNK])))  # Normalize by the maximum value

    stream.write(data)

    # Find maximum frequency and value for each range
    max_freqs = []
    max_values = []
    for freq_range, chosen_indices in zip(chosen_frequencies, [chosen_indices_1, chosen_indices_2]):
        range_values = np.abs(yf[chosen_indices]) / np.max(np.abs(yf[chosen_indices]))  # Normalize by the maximum value
        max_index = np.argmax(range_values)
        max_freq = xf[chosen_indices][max_index]
        max_value = range_values[max_index]

        max_freqs.append(max_freq)
        max_values.append(max_value)

    # Send maximum frequencies and values to Arduino
    for freq, value in zip(max_freqs, max_values):
        print(f"Frequency: {freq:.2f} Hz, Value: {value:.2f}")
        message = f"{freq:.2f}:{value}\n".encode()  # Encode message as bytes

        # Measure the time before sending the message
        start_latency = time.time()

        arduino.write(message)

        # Measure the time after receiving the message
        end_latency = time.time()

        # Calculate and store the latency
        latency = end_latency - start_latency
        latencies.append(latency)
        # Calculate the average latency
        average_latency = np.mean(latencies)
        print('Average Latency:', average_latency)

    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)

        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

    # Read the next chunk of audio data from the .wav file
    data = wav_file.readframes(CHUNK)

print('Playback finished')


stream started


error: unpack requires a buffer of 16384 bytes

KeyboardInterrupt: 

KeyboardInterrupt: 